<a href="https://colab.research.google.com/github/andrewsongj/fraud-detector/blob/main/NetSec_Classic_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
! pip install -q kaggle

from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jjg5fg","key":"e2702d4aa79e9df32c4bcd6b16b5fd12"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json



In [ ]:
!kaggle datasets download -d emilynack/aci-iot-network-traffic-dataset-2023

100% 32.3G/32.3G [20:55<00:00, 26.0MB/s]
100% 32.3G/32.3G [20:55<00:00, 27.6MB/s]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv aci-iot-network-traffic-datagset-2023.zip /content/drive/MyDrive/

mv: cannot stat 'aci-iot-network-traffic-dataset-2023.zip': No such file or directory


In [ ]:
%%capture
#!mv aci-iot-network-traffic-dataset-2023.zip /content/drive/MyDrive/
import pandas as pd
!unzip /content/drive/MyDrive/aci-iot-network-traffic-dataset-2023.zip

In [3]:
!unzip /content/drive/MyDrive/cicids2017.zip

Archive:  /content/drive/MyDrive/cicids2017.zip
  inflating: MachineLearningCSV.md5  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  


In [7]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
nRowsRead = None

df1 = pd.read_csv("MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")


df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df6])
del df6
df = pd.concat([df,df7])
del df7
df = pd.concat([df,df8])
del df8

In [8]:
features = df.columns.tolist()
features.remove(' Label')
X = df[features]
y=df[' Label']


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0, stratify=y)

In [10]:
float_columns = X_train.select_dtypes(include=['float64']).columns

In [11]:
import numpy as np

# Checking and replacing 'inf' with NaN
X_train_nan = np.where(np.isinf(X_train), np.nan, X_train)
X_test_nan = np.where(np.isinf(X_test), np.nan, X_test)

# Handling NaN values by imputing with the mean or median of the column
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')

X_train_imputed = imputer.fit_transform(X_train_nan)
X_test_imputed = imputer.transform(X_test_nan)

X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

X_train_imputed[float_columns] = X_train_imputed[float_columns].round().astype('int64')
X_test_imputed[float_columns] = X_test_imputed[float_columns].round().astype('int64')

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [13]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

In [14]:
num_features = X_train_scaled.shape
num_classes = len(np.unique(y_train_encoded))

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential([
        Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        BatchNormalization(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dense(len(encoder.classes_), activation='softmax')  # Output layer
    ])
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               20224     
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 64)                8

In [16]:
history = model.fit(X_train_scaled, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_acc}")

Epoch 1/10
42462/42462 [==============================] - 193s 4ms/step - loss: 0.0835 - accuracy: 0.9670 - val_loss: 0.0703 - val_accuracy: 0.9705
Epoch 2/10
42462/42462 [==============================] - 185s 4ms/step - loss: 0.0567 - accuracy: 0.9768 - val_loss: 0.0970 - val_accuracy: 0.9460
Epoch 3/10
42462/42462 [==============================] - 185s 4ms/step - loss: 0.0479 - accuracy: 0.9818 - val_loss: 0.0490 - val_accuracy: 0.9821
Epoch 4/10
42462/42462 [==============================] - 189s 4ms/step - loss: 0.0442 - accuracy: 0.9834 - val_loss: 0.0744 - val_accuracy: 0.9639
Epoch 5/10
42462/42462 [==============================] - 190s 4ms/step - loss: 0.0431 - accuracy: 0.9837 - val_loss: 0.0647 - val_accuracy: 0.9794
Epoch 6/10
42462/42462 [==============================] - 190s 4ms/step - loss: 0.0464 - accuracy: 0.9820 - val_loss: 0.0596 - val_accuracy: 0.9794
Epoch 7/10
42462/42462 [==============================] - 189s 4ms/step - loss: 0.0423 - accuracy: 0.9842 - val_

In [17]:
model.save('/content/drive/MyDrive/my_network_model_reversed.h5')  # Saves as a single HDF5 file

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
%%capture
#!mv aci-iot-network-traffic-dataset-2023.zip /content/drive/MyDrive/
import pandas as pd
!unzip /content/drive/MyDrive/aci-iot-network-traffic-dataset-2023.zip

In [6]:
df = pd.read_csv("ACI-IoT-2023.csv")

In [7]:
df.drop(["Flow Bytes/s","Timestamp","Flow Packets/s"],axis=1,inplace=True)

In [8]:
#selected_labels = ['ICMP Flood', 'Slowloris', 'SYN Flood', 'UDP Flood', 'DNS Flood', 'Benign']
#df =df[df['Label'].isin(selected_labels)]

In [9]:
features = df.columns.tolist()
features.remove("Label")
features.remove("Flow ID")
features.remove("Src IP")
features.remove("Dst IP")
X = df[features]
y=df['Label']

In [10]:
import pandas as pd
X = pd.get_dummies(X, columns=["Connection Type"],dtype=int)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0, stratify=y)
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [14]:
from tensorflow.keras.models import load_model
import numpy as np
base_model = tf.keras.models.load_model('/content/drive/MyDrive/my_network_model_reversed.h5')

num_features = X_train.shape
num_classes = len(np.unique(y_train_encoded))

In [15]:
for layer in base_model.layers[:-1]:
    layer.trainable = False

# Print the trainable status of each layer
for layer in base_model.layers:
    print(layer, layer.trainable)

<keras.src.layers.core.dense.Dense object at 0x7d44922d7730> False
<keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7d4493293e80> False
<keras.src.layers.core.dense.Dense object at 0x7d44922d7be0> False
<keras.src.layers.regularization.dropout.Dropout object at 0x7d44922d7bb0> False
<keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7d44922d6650> False
<keras.src.layers.core.dense.Dense object at 0x7d44922d6920> False
<keras.src.layers.core.dense.Dense object at 0x7d44922d7dc0> True


In [19]:
from tensorflow.keras.layers import Dense


base_model.pop()  # Removes the last layer
base_model.add(Dense(num_classes, activation='softmax'))  # Add a new output layer

In [20]:
from tensorflow.keras.models import Sequential
new_model = Sequential()
new_model.add(Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)))


# Add all other layers from the original model except the input
for layer in base_model.layers[1:]:
    new_model.add(layer)


In [21]:
for layer in new_model.layers[:-1]:  # Freeze all but the new output layer
    layer.trainable = False

In [23]:
from tensorflow.keras.optimizers import Adam

# Recompile the model
new_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model (only the unfrozen layers will be updated)
history = new_model.fit(X_train_scaled, y_train_encoded, epochs=50, batch_size=500, validation_split=0.2)
test_loss, test_acc = new_model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_acc}")

Epoch 1/50
1183/1183 [==============================] - 4s 3ms/step - loss: 0.6962 - accuracy: 0.7947 - val_loss: 0.5515 - val_accuracy: 0.8234
Epoch 2/50
1183/1183 [==============================] - 3s 3ms/step - loss: 0.4956 - accuracy: 0.8399 - val_loss: 0.4461 - val_accuracy: 0.8536
Epoch 3/50
1183/1183 [==============================] - 3s 3ms/step - loss: 0.4178 - accuracy: 0.8615 - val_loss: 0.3901 - val_accuracy: 0.8802
Epoch 4/50
1183/1183 [==============================] - 3s 3ms/step - loss: 0.3721 - accuracy: 0.8910 - val_loss: 0.3529 - val_accuracy: 0.9100
Epoch 5/50
1183/1183 [==============================] - 3s 3ms/step - loss: 0.3409 - accuracy: 0.9107 - val_loss: 0.3266 - val_accuracy: 0.9212
Epoch 6/50
1183/1183 [==============================] - 3s 3ms/step - loss: 0.3177 - accuracy: 0.9226 - val_loss: 0.3066 - val_accuracy: 0.9277
Epoch 7/50
1183/1183 [==============================] - 3s 3ms/step - loss: 0.2997 - accuracy: 0.9277 - val_loss: 0.2913 - val_accuracy:

In [24]:
#!kaggle datasets download -d azalhowaide/iot-dataset-for-intrusion-detection-systems-ids

In [25]:
%%capture
#!mv iot-dataset-for-intrusion-detection-systems-ids.zip /content/drive/MyDrive/
#import pandas as pd
!unzip /content/drive/MyDrive/iot-dataset-for-intrusion-detection-systems-ids.zip

In [26]:
iot = pd.read_csv('BoTNeTIoT-L01-v2.csv')

In [27]:
iot.columns

Index(['MI_dir_L0.1_weight', 'MI_dir_L0.1_mean', 'MI_dir_L0.1_variance',
       'H_L0.1_weight', 'H_L0.1_mean', 'H_L0.1_variance', 'HH_L0.1_weight',
       'HH_L0.1_mean', 'HH_L0.1_std', 'HH_L0.1_magnitude', 'HH_L0.1_radius',
       'HH_L0.1_covariance', 'HH_L0.1_pcc', 'HH_jit_L0.1_weight',
       'HH_jit_L0.1_mean', 'HH_jit_L0.1_variance', 'HpHp_L0.1_weight',
       'HpHp_L0.1_mean', 'HpHp_L0.1_std', 'HpHp_L0.1_magnitude',
       'HpHp_L0.1_radius', 'HpHp_L0.1_covariance', 'HpHp_L0.1_pcc',
       'Device_Name', 'Attack', 'Attack_subType', 'label'],
      dtype='object')

In [28]:
features = iot.columns.tolist()
features.remove('label')
features.remove('Attack')
features.remove('Attack_subType')
features.remove('Device_Name')
X = iot[features]
y=iot['Attack']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)


In [30]:
float_columns = X_train.select_dtypes(include=['float64']).columns
# Checking and replacing 'inf' with NaN
X_train_nan = np.where(np.isinf(X_train), np.nan, X_train)
X_test_nan = np.where(np.isinf(X_test), np.nan, X_test)

# Handling NaN values by imputing with the mean or median of the column
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')  # or strategy='median'

X_train_imputed = imputer.fit_transform(X_train_nan)
X_test_imputed = imputer.transform(X_test_nan)

X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

X_train_imputed[float_columns] = X_train_imputed[float_columns].round().astype('int64')
X_test_imputed[float_columns] = X_test_imputed[float_columns].round().astype('int64')


In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [32]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

In [33]:
num_features = X_train_scaled.shape
num_classes = len(np.unique(y_train_encoded))

In [34]:
from tensorflow.keras.models import load_model
base_model = tf.keras.models.load_model('/content/drive/MyDrive/my_network_model_reversed.h5')

In [35]:
from tensorflow.keras.layers import Dense


base_model.pop()  # Removes the last layer
base_model.add(Dense(num_classes, activation='softmax'))  # Add a new output layer

from tensorflow.keras.models import Sequential
new_model = Sequential()
new_model.add(Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)))


# Add all other layers from the original model except the input
for layer in base_model.layers[1:]:
    new_model.add(layer)


In [36]:
for layer in new_model.layers[:-1]:  # Freeze all but the new output layer
    layer.trainable = False

In [38]:
from tensorflow.keras.optimizers import Adam

# Recompile the model
new_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model (only the unfrozen layers will be updated)
history = new_model.fit(X_train_scaled, y_train_encoded, epochs=50, batch_size=500, validation_split=0.2)
test_loss, test_acc = new_model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_acc}")

Epoch 1/50
9041/9041 [==============================] - 29s 3ms/step - loss: 0.6347 - accuracy: 0.7226 - val_loss: 0.5303 - val_accuracy: 0.8362
Epoch 2/50
9041/9041 [==============================] - 27s 3ms/step - loss: 0.6349 - accuracy: 0.7228 - val_loss: 0.5300 - val_accuracy: 0.8363
Epoch 3/50
9041/9041 [==============================] - 30s 3ms/step - loss: 0.6346 - accuracy: 0.7227 - val_loss: 0.5309 - val_accuracy: 0.8352
Epoch 4/50
9041/9041 [==============================] - 27s 3ms/step - loss: 0.6348 - accuracy: 0.7231 - val_loss: 0.5293 - val_accuracy: 0.8367
Epoch 5/50
9041/9041 [==============================] - 27s 3ms/step - loss: 0.6344 - accuracy: 0.7232 - val_loss: 0.5284 - val_accuracy: 0.8371
Epoch 6/50
9041/9041 [==============================] - 27s 3ms/step - loss: 0.6346 - accuracy: 0.7232 - val_loss: 0.5281 - val_accuracy: 0.8377
Epoch 7/50
9041/9041 [==============================] - 27s 3ms/step - loss: 0.6341 - accuracy: 0.7233 - val_loss: 0.5289 - val_ac